# Извлечение имен собственных из текстов (описаний сборов) с помощью DeepPavlov
## Вирутальная среда
Скрипт написан под python 3.7., т.к. на момент его написания DeepPavlov не работал на 3.8. Чтобы устновить другую версию Python и указывать ее как интерпретатор для кода, создана виртуальная среда. В правом верхн. углу ноутбука в Jupyter отображается my_new_env (название вирт. среды) как кернель. Шпаргалка, как работать с вирт. средой. В терминале:

```
$ virtualenv -p python3.7 my_new_env # указываем, что в этой среде нужна эта версия питона
$ source my_new_env/bin/activate # активируем среду```

[Дальше: как создать ноутбук и указать кернель](https://janakiev.com/blog/jupyter-virtual-envs/#add-virtual-environment-to-jupyter-notebook)

## DeepPavlov
В этом ноутбуке используются модели DeepPavlov для NER для русского языка. [Другие модели доступны здесь.](http://docs.deeppavlov.ai/en/master/features/models/ner.html) 

In [42]:
import sys
sys.executable # проверяем, что находимся в нужном окружении (среде)

'/Users/liza/my_new_env/bin/python'

In [5]:
!python -m deeppavlov install ner_rus # установка моделей для русского языка.
!python -m deeppavlov install ner_rus_bert

2020-06-04 21:16:31.689 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'ner_rus' as '/Users/liza/my_new_env/lib/python3.7/site-packages/deeppavlov/configs/ner/ner_rus.json'
2020-06-04 21:16:35.333 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'ner_rus_bert' as '/Users/liza/my_new_env/lib/python3.7/site-packages/deeppavlov/configs/ner/ner_rus_bert.json'
  Cloning https://github.com/deepmipt/bert.git (to revision feat/multi_gpu) to /private/var/folders/c2/gzfqq3mn3813qp4lsy9gllcw0000gn/T/pip-req-build-b3febsdm
  Running command git clone -q https://github.com/deepmipt/bert.git /private/var/folders/c2/gzfqq3mn3813qp4lsy9gllcw0000gn/T/pip-req-build-b3febsdm
  Running command git checkout -b feat/multi_gpu --track origin/feat/multi_gpu
  Switched to a new branch 'feat/multi_gpu'
  Branch 'feat/multi_gpu' set up to track remote branch 'feat/multi_gpu' from 'origin'.
  Created wheel for bert-dp: filename=bert_dp-1.0-py3-none-any.whl size=23580 

In [ ]:
help('modules') # проверяем, что все установилось и доступно в этой среде

In [2]:
import pandas as pd
from nltk.tokenize import sent_tokenize
from rusenttokenize import ru_sent_tokenize # модель DP для токенизации на предложения, по их тестам она показывает рез. лучше, чем nltk
import re
import time # для time.sleep

In [270]:
# Строки для промежуточных тестов токенизации и др. функции (выдуманные и из датасета):

text1 = '''
Кирилл тяжело болен с двух лет. Кирюше нужна операция. Он очень людит своего кота. Не просиьт же Владимира Путина 
Давайте поможем Кирюне встать на ноги. Пусть Кирюшке снова повезет'''

text2 = '''Цель проекта - ликвидировать долги за ветеринарные услуги, накопившиеся в январе 2017 года за 
лечение, рентген, анестезию, УЗИ, анализы, инъекции питомцам "Островка надежды" г.Пермь
И снова здравствуйте, Друзья!
Меня зовут Надежда Николаевна, я директор Местной общественной организации защитников животных 
"Островок надежды" г.Пермь, мы называем себя Центром реабилитации для собак "Островок надежды" НЕ 
являемся муниципальным приютом и не получаем бюджетных денежных средств.
История нашего Островка началась в 2010 году. В настоящее время в Островке нашли свой дом 450 собак и 
бывшедомашний, особенный кот Кипиш , который после случившейся трагедии в 2017 году стал инвалидом и 
находится в стационаре на лечении.
Все питомцы Островка, в т. ч. особенные и возрастные собаки живут и получают ветеринарную помощь с первого до последнего дня и проживают до тех пор пока не найдут своего хозяина.
В этой жизни их никто не ждал, но они выжили и с первых дней, как могли боролись за нее. Сейчас все иначе, 
теперь мы вместе боремся за каждого питомца. Это становится возможным благодаря помощи, поступающей от 
благотворителей, помощников, волонтеров, всех неравнодушных Пермяков и конечно Вашей неоценимой помощи, 
Друзья. Цель нашего проекта "Не вешай нос! Или право на жизнь" - ликвидировать долг за ветеринарные услуги, 
накопившиеся в январе и начале февраля 2017 года.
С тяжёлыми травмами, обмороженные, голодные, с множеством болезней, животные попадают в приют. 
Мы - немногочисленная команда волонтеров и сотрудники приюта, действительно всеми силами отвоевываем 
каждого питомца у смерти, возвращая им веру в человечность людей.
Сегодня мы Вас знакомим с 9 нашими питомцами, которым уже с первых дней начавшегося 2017 года потребовалась 
неотложная ветеринарная помощь. В январе средств не было, но и поступить по-другому мы не имели право. 
Лечили в долг. Спасибо Клинике "Ветлайн" за сотрудничество, за предоставление скидок на лечение наших 
питомцев, за оказание необходимой помощи в долг.
Девять Островитян нуждаются в оплате за лечение, проведенные процедуры, рентген, анастезию, УЗИ, анализы, 
инъекции... Сегодня их жизни уже ничего не угрожает, лечение продолжается, но образовавшийся долг остался.
Маня Печенька
Хрюня Рекс|
Люся
Джина Маша
Студент
Лаки
Тайга
В январе спасен кот Кипиш (кота разорвали собаки, Кипиш потерял переднюю лапку), в настоящее время 
находится в стационаре на лечении.
Лечение дорогостоящее, не судите нас, но мы ни на минуту не сомневались, что кота нужно спасать, силы и 
средства брошены на спасение маленькой, хрупкой жизни Кипиша.
Документы, подтверждающие расходы:
Вы были с нами каждый Проект, поддерживали и помогали.
Мы боль сердец свою несли, и Вы нас выручали.
Не останьтесь в стороне! Примите участие в Проекте!
Всегда Ваши "Островок Надежды" г. Пермь'''

text3 = '''Полные отчеты о реализации проекта и фото из жизни наших подопечных можно всегда посмотреть в нашей группе в социальной сети ВКонтакте
http://vk.com/shanszoopskov.и на нашей странице в facebook.https://www.facebook.com/shanszoopskov/.
Публикации о нас в прессе:.Начало работы АНЗО "Шанс".http://www.gtrkpskov.ru/news-feed/news/9581-novaya....
О контракте на отлов безнадзорных животных в городе Пскове.http://www.gtrkpskov.ru/news-feed/news/10172-obshc....'''

text4 = '''Друзья! Благотворительный Фонд «Разные Люди – Рок-музыканты помогают детям» и народный портал Planeta.Ru запускает новый проект по сбору средств на лечение и реабилитацию подопечных Фонда «Разные Люди». Историю Фонда и его проектов - Рок-Фестивалей и Рок-Аукционов можно посмотреть на сайте https://www.raznyeludi.ru/help. Там же можно ознакомиться с финансовым отчётом о работе Фонда. Новый проект носит название «АЛЕКСАНДР ЧЕРНЕЦКИЙ – ТОМ III. АКУСТИКА», собранные средства от продаж которого будут использоваться для оплаты лечения и реабилитации подопечных фонда.
   Сама Антология являнтся наглядным примером того, что люди с ограниченными возможностями могут вести активный образ жизни - исполнять и записывать свою музыку, участвовать в концертах, гастролировать по миру. Что безусловно является примером борьбы с недугом и полноправной ассоциацией в обществе. Данные записи собраны Александром, отреставрированны и подготовлены к изданию его друзьями. Записи отражают период творчества Александра Чернецкого с 1982 по 2016 годы. Ранее не издавались. Проэкт преследует гуманитарные, просветительские и безусловно, благотворителые цели.
Проект завершается 1 июня 2018 года в День Защиты Детей.
Участвуйте в Благотворительности вместе с нами!
    Список аудио-записей «Том III. Акустика»:
1982 -1984. Саша Чернецкий – «Домашние записи» (Харьков)
1983. Саша Чернецкий & Юра Тверитников – «Концерт у Иры Чернявской» (июнь 1983, Харьков)
1987. Чернецкий & СЭР – «Концерт у Мясоедовых» (07. 03. 1987, Харьков)
1987. Разные Люди – «Фестиваль – Рига-87» 2 концерта (10 - 12. 07. 1987, ДК Страуме, Рига)
1987. Разные Люди – «Конец июля» (июль 1987, Харьков)
1988. Александр Чернецкий – «Акустика в Пятихатках» (19. 03. 1988, Харьков)
1988. Александр Чернецкий – «Квартирник на Кацарской. Песни 1982-1988» (май 1988, Харьков)
1988. КПСС – «Концерт в «Юности» (20. 12. 88, Харьков)
1989. Разные Люди – «Рождество в Звановке» (07. 01. 1989, Звановка)
1988. Александр Чернецкий – «Боль» (сентябрь 1988, Ленинград)
1989. Александр Чернецкий - «Вера» (май 1989, Харьков)
1989. Сашка, Янка, Коча – «Сейшн у Чернецкого» (26. 06. 89, Харьков)
1991. Александр Чернецкий – «С Наташкиной гитарой» (ноябрь 1991, Питер)
1992. Александр Чернецкий – «Больница Святого Георгия» (январь 1992, Питер)
1992. Александр Чернецкий – «Не Было (акустический альбом, Харьков)
1992. Разные Люди – «На кухне у Чернецкого (03. 08. 1992, Харьков)
1993. Александр Чернецкий – «Кружка Эсмарха» (акустический альбом, Харьков)
1993. Разные Люди – «Проводы Кошмара в Америку» (01. 04. 1993, ХГУ, Харьков)
1993. Александр Чернецкий – «Музейник в «От Винта!» (01. 10. 1993, Харьков)
1994. Сашка и Клим – «У Пети в квартире 14» (26. 03. 1994, Харьков)
1994. Разные Люди – «Vox-Club» (15. 04. 1994, Москва)
1994. Александр Чернецкий – «Музейник в «От Винта!» (16. 05. 1994, Харьков)
1994. Александр Чернецкий – «Сходка авторитетов в «От Винта!» (04. 06. 1994, Харьков)
1994. Чернецкий и Клим – «Квартирник у Алеса» (июль 1994, Москва)
1994. Александр Чернецкий – «Концерт в М.Р.С» (13. 10. 1994, Харьков)
1994. Разные Люди – «Фестиваль «Аку Рок Два» (30. 10. 94, Москва)
1997. Александр Чернецкий – «Концерт в КЛРР» (21. 11. 1997, Харьков)
1997. Александр Чернецкий – «Концерт на Рубинштейна, 13» (апрель 1997, Питер)
1998. Александр Чернецкий – «Концерт в КлрР» (03. 01. 1998, Харьков)
1998. Александр Чернецкий – «Концерт в УКЦ «Юность» (21. 02. 1998, Харьков)
1998. Александр Чернецкий – «Концерт в клубе «Паутина» (Харьков, ДС ХПИ 22. 09. 1998 г.)
1999. Александр Чернецкий – «Канун 33-х» (09. 01. 1999, квартирник у А. Мирошниченко, Харьков)
1999. Александр Чернецкий и Пётр Белецкий – «Концерт в КЛРР» (8. 04. 1999, Харьков)
1999. Александр Чернецкий – «Концерт в ДНК» (31. 03. 1999, Харьков)
1999. Черненцкий & Сланч – «Концерт в КЛРР» (22. 05. 1999, Харьков)
1999. Александр Чернецкий – «Жизнь» (Радио-Ютар, июнь 1999, Киев)
2000. Чернецкий и Кожекин – «Форпост» (07. 01. 2000, Москва)
2000. Чернецкий & Кожекин – Концерт в «Факеле» (08. 01. 2000)
2000. Чернецкий и Чиж – «Концерт в «Зоопарке» (2. 02. 2000, Питер)
2000. Александр Чернецкий – «Квартирник в Питере» (05. 05. 2000)
2000. Александр Чернецкий в КлрР (14. 09. 2000, Харьков)
2003. Александр Чернецкий – «Концерт памяти Кати Гуцал» (Питер)
2004. Чернецкий и Чиж в музее Блока \ Че и Чи в музее Блока (27.03.2004)
2004. Разные Люди - Презентация альбома «АКУСТИКА» (Москва, ЦДХ, 02.06.04)
2005. Александр Чернецкий – «Концерт в Нью-Йорке» (21. 10. 05, NYC)
2005. Александр Чернецкий - «Концерт в Сан-Франциско» (28. 10. 05, SF)
2006. Разные Люди - Квартирник на О2ТВ
2007. Чернецкий & Михайленко – «Концерт в «Камчатке» (23. 09. 2007, Питер)
2008. Чернецкий & Акимов - Концерт с виолончелью (Москва, Клуб «Жесть» 28. 08. 2008)
2011. Александр Чернецкий – «Концерт в Нижнем Новгороде» (23. 01. 2011)
2012. Разные Люди – «Концерт в Пожарке» (19. 04. 2012, Москва)
2013. А. Чернецкий – «Презентация диска «Разные Люди в Америке» в магазине «Дом Культуры» (3. 03. 2013, Москва)
2013. Александр Чернецкий. Квартирник на Павелецкой 7.02.2013
2014. Чернецкий & Васильев – «Фестиваль ГАРКУНДЕЛЬ»
2016. Разные Люди - Unplugged на Лиговке 14.10.16
***** Радио-эфиры, интервью (mp3) *********
1990. Александр Чернецкий – Интервью для «Зеро Поверх» (Харьков)
1995. Разные Люди – Концерт на «Радио-50» (28. 01. 1995,Харьков)
1997. Александр Чернецкий – Концерт на Радио-1 Петроград
1999. Александр Чернецкий – Интервью на «Радио-Ютар» (Киев)
2002. Александр Чернецкий – на радио «Шансон» (Питер)
2005. Чернецкий & Махов – на радио «Шансон» (27. 08. 2005, Харьков)
2005. Александр Чернецкий - В Нью-Йорке с Виктором Топаллером (11. 11. 2005, NYC)
2006. Александр Чернецкий – Интервью для газеты «Марата 27» (Питер)
2008. Александр Чернецкий – Эфир на радио «Советский спорт» (24. 10. 2008, Москва)
2010. Разные Люди – на «Radio Rock On-Line» (16. 10. 2010, Питер)
2011. Александр Чернецкий – на «Radio Pozitiv 87.7FM» (15. 06. 2011, NYC, USA)
  Лоты для вознаграждения:
1. Александр Чернецкий. Чёрный ворон - я не твой. Том III. Акустика - 5 000 р.
2. Полное издание 1-го благотворительного фестиваля «Разные Люди» с автографами Шевчука, Чижа, Чернецкого - 20 000 р.
3. Александр Чернецкий. Чёрный ворон - я не твой. Том II. Электричество - 5 000 р.
4. Александр Чернецкий - Чёрный ворон - я не твой! Том I. Студия - 3 000 р.
5. Фирменная футболка 1 и 2-го благотворительных фестивалей «Разные Люди» - 2 000 р.
6. Фирменная футболка 3-го благотворительного фестиваля «Разные Люди» - 2 000 р.
7. Футболка группы «Разные Люди» (дизайн художника «DDT» Владимира Дворника) - 1 000 р.
8. РАЗНЫЕ ЛЮДИ & ЧИЖ & Co - ОДИН ДЕНЬ ВМЕСТЕ. 2007 © (4 CD, Slipcase Box) - 3 000 р.
9. ЧЕРНЕЦКОМУ – 50! РАЗНЫЕ ЛЮДИ, ЧИЖ & CO, ДЕКАБРЬ, ЧЧ (2 CD + 2 DVD) - 2 500 р.
10. ЧЕРНЕЦКОМУ – 50! РАЗНЫЕ ЛЮДИ, ЧИЖ & CO, ДЕКАБРЬ, ЧЧ (Blu-Ray 50 Gb) - 1 500 р.
11. Александр Чернецкий - Страна Престижа © 2010 (2 CD, Slipcase Box, 2 book Х 40 p) - 1500 р.
12. РАЗНЫЕ ЛЮДИ - 911 © 2003 (CD, Slipcase Box,book 16 p). С автографами Чижа и Чернецкого - 1 500 р.
13. РАЗНЫЕ ЛЮДИ - ЧЕРНЕЦ © 2013 (CD, digipak, book 16 p). С автографом Чернецкого 1 500 р.
14. ГПД - РАЗНЫЕ ЛЮДИ. АНТОЛОГИЯ © 1987 - 1997 (CD mp3) С автографами Чижа и Чернецкого- 1 000 р.
15. РАЗНЫЕ ЛЮДИ - 1992 © 1992 (CD). С автографами Чижа и Чернецкого - 1 000 р.
16. ЧЕРНЕЦКИЙ & ЧИЖ - COMEBACK © 2000 (CD). С автографами Чижа и Чернецкого - 1 000 р.
17. РАЗНЫЕ ЛЮДИ - ДОРОГИ © 2008 (CD). С автографами Чижа и Чернецкого - 1 000 р.
18. Разные Люди - 15 и 25-летие группы © 2014 (3 DVD, digipak). С автографом Чернецкого - 1000 р.
19. Разные Люди - 25 лет! Белгород ЦМИ © 2014 (2 DVD-5 + 9). С автографом Чернецкого - 500р.
20. Разные Люди - ХХ лет рок-н-ролла (DVD). С автографом Чернецкого - 400 р.
21. РАЗНЫЕ ЛЮДИ - РАЗНЫЕ ПЕСНИ © 2005 (CD). С автографами Чижа и Чернецкого - 1 000 р.
22. Разные Люди - Мазохизм © 1990 (CD). С автографами Чижа и Чернецкого - 1 000 р.
23. РАЗНЫЕ ЛЮДИ - Презентация LP 1992 + Программа А (DVD). С автографом Чернецкого - 400 р.
24. РАЗНЫЕ ЛЮДИ. Антропология (1998) - Живая коллекция (1997) (DVD). С автографом Чернецкого - 400 р.
25. РАЗНЫЕ ЛЮДИ - В программе Антропология (2000) (DVD). С автографом Чернецкого - 400 р.
26. РАЗНЫЕ ЛЮДИ - В программе «Земля - Воздух» (2002) (DVD). С автографом Чернецкого - 400р.
27. РАЗНЫЕ ЛЮДИ. Презентация «911» (ЦДХ. 25. 02.04) (DVD). С автографом Чернецкого - 400 р.
28. Че и Чи в музее Блока (27.03.04) (DVD). С автографом Чернецкого - 400 р.
29. РАЗНЫЕ ЛЮДИ. Презентация «АКУСТИКА» (ЦДХ. 02. 06. 04) (DVD). С автографом Чернецкого - 400 р.
30. РАЗНЫЕ ЛЮДИ - В программе Чистый звук, 5 канал, Санкт-Петербург (2004) (DVD). С автографом Чернецкого - 400 р.
31. РАЗНЫЕ ЛЮДИ - В программе В Нью-Йорке с Виктором Топаллером (Гость А.Чернецкий). Чиж и Чернецкий - В программе ЧУЖАЯ КУХНЯ (2006) (DVD). С автографом Чернецкого - 400 р.
32. РАЗНЫЕ ЛЮДИ - ДР Высоцкого (ЦДЖ. 21. 01. 04) (DVD). С автографом Чернецкого - 400 р.
33. РАЗНЫЕ ЛЮДИ - Квартирник на О2ТВ (2006) (DVD). С автографом Чернецкого - 400 р.
34. Чернецкий & Акимов - Клуб «Жесть» (2008) (DVD). С автографом Чернецкого - 400 р.
35. Полковник и Однополчане - Чернецкий и Разные Люди в клубе «Зоопарк» (2000) (DVD). С автографом Чернецкого - 400 р.
36. Чернецкий в Калифорнии (11. 11. 2009) (DVD). С автографом Чернецкого - 400 р.
37. РАЗНЫЕ ЛЮДИ. Окна Открой!, Крылья, Воздух, Майку-45, Высоцкий (2000 - 2009) (DVD). Савтографом Чернецкого - 400 р.
38. РАЗНЫЕ ЛЮДИ - TV-эфиры. Видеоклипы 1990 - 2010 (DVD). С автографом Чернецкого - 400 р.
39. РАЗНЫЕ ЛЮДИ - Презентация альбома ДОРОГИ (День первый. Vol.2) (Mezzo Forte, 24.10.08)(DVD). С автографом Чернецкого - 400 р.
40. РАЗНЫЕ ЛЮДИ - Презентация альбома ДОРОГИ, Трамплин (Москва) День 2-ой (25.10.2008)(DVD). С автографом Чернецкого - 400 р.
41. РАЗНЫЕ ЛЮДИ - Концерт с Иваном Васильевым (Орландина, 15.01.09) (DVD). С автографом Чернецкого - 400 р.
42. РАЗНЫЕ ЛЮДИ - Фестиваль «КРАСНОЕ НА ЧЕРНОМ» (Харьков, 29.05.10) (DVD). С автографомЧернецкого - 400 р.
43. РАЗНЫЕ ЛЮДИ - «РОЖДЕННЫЕ В СССР» (NOCTALЬГИЯ, 12. 01. 11) (DVD). С автографом Чернецкого - 400 р.
44. РАЗНЫЕ ЛЮДИ - Зал Ожидания. Санкт-Петербург, 21.01.2011. День Рождения Александра Чернецкого и Михаила Нефёдова (DVD). С автографом Чернецкого - 400 р.
45. РАЗНЫЕ ЛЮДИ - Нашествие - 2011. Окна Открой! - 2011. Сборник 1991 - 2011 (DVD). С автографом Чернецкого - 400 р.
46. РАЗНЫЕ ЛЮДИ - Юбилей в «Mezzo Forte» 22.01.11 (DVD). С автографом Чернецкого - 400 р.
47. РАЗНЫЕ ЛЮДИ в программе «OLIV.E» (Харьков, 2011) (DVD). С автографом Чернецкого - 400р.
48. РАЗНЫЕ ЛЮДИ - Чиж & Со - Юбилейный концерт (ЦМИ, Белгород 7.10.11) (DVD). Савтографом Чернецкого - 400 р.
49. РАЗНЫЕ ЛЮДИ - 23 февраля. День Советской Армии и Военно-Морского Флота. 2012 (DVD). С автографом Чернецкого - 400 р.
50. Разные Люди - Фестивали. Лето 2012 (DVD). С автографом Чернецкого - 400 р.
51. Разные Люди - Концерт в пожарке (Москва, 19.04.12) (DVD). С автографом Чернецкого - 400р.
52. Разные Люди - Клуб Jazzter (Харьков, 13.10.12) (DVD). С автографом Чернецкого - 400 р.
53. Группа Продленного Дня (ГПД). VI Ленинградский рок-фестиваль. Зимний стадион. 10 июня 1988 (DVD). С автографом Чернецкого - 400 р.
54. Разные Люди / ХА.МЫ. - «Проводы Худого» (Москва, клуб «Меццо Форте», 16. 11. 12) (DVD). С автографом Чернецкого - 400 р.
55. Разные Люди - 25 лет! Гости - ДДТ, Ангел Небес, Ливень (Питер, «Зал Ожидания», 9. 04. 2014)(DVD). С автографом Чернецкого - 400 р.
56. Разные Люди - 25 лет! Москва 11 апреля 2014 (DVD). С автографом Чернецкого - 400 р.'''

In [6]:
from deeppavlov import configs, build_model 
ner = build_model(configs.ner.ner_rus_bert, download=True) # собирает модель для NER

2020-06-04 21:16:54.855 INFO in 'deeppavlov.download'['download'] at line 117: Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v1.tar.gz download because of matching hashes
2020-06-04 21:16:58.377 INFO in 'deeppavlov.download'['download'] at line 117: Skipped http://files.deeppavlov.ai/deeppavlov_data/ner_rus_bert_v1.tar.gz download because of matching hashes
2020-06-04 21:16:58.801 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /Users/liza/.deeppavlov/models/ner_rus_bert/tag.dict]
2020-06-04 21:17:30.788 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /Users/liza/.deeppavlov/models/ner_rus_bert/model]


INFO:tensorflow:Restoring parameters from /Users/liza/.deeppavlov/models/ner_rus_bert/model


In [13]:
ru_sent_tokenize(text2) # проверяем работоспособность токенизатора DP

['Цель проекта - ликвидировать долги за ветеринарные услуги, накопившиеся в январе 2017 года за \nлечение, рентген, анестезию, УЗИ, анализы, инъекции питомцам "Островка надежды" г.Пермь\nИ снова здравствуйте, Друзья!',
 'Меня зовут Надежда Николаевна, я директор Местной общественной организации защитников животных \n"Островок надежды" г.Пермь, мы называем себя Центром реабилитации для собак "Островок надежды" НЕ \nявляемся муниципальным приютом и не получаем бюджетных денежных средств.',
 'История нашего Островка началась в 2010 году.',
 'В настоящее время в Островке нашли свой дом 450 собак и \nбывшедомашний, особенный кот Кипиш , который после случившейся трагедии в 2017 году стал инвалидом и \nнаходится в стационаре на лечении.',
 'Все питомцы Островка, в т. ч. особенные и возрастные собаки живут и получают ветеринарную помощь с первого до последнего дня и проживают до тех пор пока не найдут своего хозяина.',
 'В этой жизни их никто не ждал, но они выжили и с первых дней, как могли 

## Создаем все нужные функции

### Комментарий к функции для предобработки текста
Он нужна, в первую очередь, для токенизации текста перед подачей его в ner-модель. Без этого модель не справится. Сама модель разбивает текст на субтокены так, как это делает BERT (DP основан на нем). BERT использует WordPiece tokenizer и имеет ограничение, которое выливается в ошибку, если текст не разбить на предложения до подачи: 
> input sequence after bert tokenization shouldn't exceed 512 tokens.

[Объяснение с гитхаба:](https://github.com/deepmipt/DeepPavlov/issues/839)

> Sorry, but the BERT model has positional embeddings only for first 512 subtokens. So, the model can't work with longer sequences. It is a deliberate architecture restriction. Subtokens are produced by WordPiece tokenizer (BPE). 512 subtokens correspond approximately to 300-350 regular tokens for multilingual model. Make sure that you performed sentence tokenization before dumping the data. Every sentence in the dumped data should be separated by an empty line.

Под regular tokens, по всей видимости, подразумеваются слова. Еще одна проблема состоит в том, что даже после разбиения на предложения (правилами), а потом на субтокены (внутри модели), последовательность получается больше 512 субтокенов. Опытным путем удалось выяснить, что в данном датасете это чаще всего происходит из-за больших списков, где строки разбиваются переносом без точки. 

In [7]:
# функция для предобработки текста перед подачей в модель для NER 

def sent_split(atext):
    atext = atext.strip() # иногда в самом конце всего текста стоит пробел (или пустая строка) после точки, который становистя пустым токеном, что потом создает проблемы 
    atext = re.sub(r"\n|\r", ". ", atext) # меняем переносы строк на точку, т.к. иногда авторы делают список, разбивают строки через enter и не ставят точку в конце фразы
    #atext = re.sub(r"http\S*", "", atext) # иногда ссылки бывают очень длинные и без знаков препинания между ними, это создает проблемы при токенизации (получается гигантский токен)
    sents = ru_sent_tokenize(atext) # токенизатор DeepPavlov, с помощью # можно переключаться с него на nltk
    #sents = sent_tokenize(atext)  # токенизатор nltk
    
    return(sents) # возвращает список предложений

In [339]:
f = sent_split(text10) # проверка функиции предобработки на случайной строке
f

['Запах костра, свет солнца сквозь ткань палатки, звон комаров, еда с дымком, гитарные переборы...',
 'Маша очень хочет в поход.',
 'Летний поход - это друзья, красивые места, купание, ягоды с куста и море впечатлений на весь следующий год.',
 'А если в семье растет ребенок с тяжелыми нарушениями, то поход для него - это еще и терапия..',
 'Отсутствие стен, потолков, ровного асфальта для детей с ограниченными возможностями, часто запертых в городских квартирах, - это разрыв шаблона, преодоление стереотипов, а значит - расширение возможностей взаимодействия с миром.',
 'Ведь в походе будут и неровные тропинки, и незнакомые запахи, звуки и новая еда, и другой распорядок дня..',
 'Уже не первый год семьи из межрегиональной общественной организации помощи детям с особенностями психоречевого развития "Дорога в мир" (http://www.dorogavmir.ru) отправляются в ПОХОД.',
 'Мы с детьми разбиваем лагерь недалеко от Оки в сосновом лесу.',
 'Педагоги вместе с ребятами исследуют местность, ищут источн

### Комментарий к функции извлечения имен
Ключевая часть здесь *ner([item])*. Переменная *ner* создана выше и содержит саму модель для извлечения. Изначально она принимает предложение (токен-строку), уложенное в одноуровневый список (просто строку не принимает, будет ошибка). Чтобы подать несколько предложений, каждое предложение должно быть упаковано в список, и все они - еще в один общий, по которому нужно проходить циклом. Или в самом цикле, как сделано в этом ноутбуке, подавая в модель эелементы большого списка, укладывать их в [] (ner[item])).  Ср.:

```ner(["Мама мыла раму"],['Шла Саша по шоссе'])
Out: [[['Мама', 'мыла', 'раму']], [['O', 'O', 'O']]]```


```ner([["Мама мыла раму"],['Шла Саша по шоссе']])
Out: [[['Мама мыла раму'], ['Шла Саша по шоссе']], [['O'], ['O']]]```

В первом случае обрабатывается только первый элемент. Во втором (без цикла) предложение воспринимается как единый токен.

В реализации ниже модель принимает список предложений (предварительно токенизированный текст), каждое предложение укладывает в список и для каждого возвращает два двухуровневых списка: слова и теги. 
```tokens, tags = ner(["Шла Саша Соколова по шоссе в Краснодаре мимо театра имени Чехова"])
Out: 
[['Шла', 'Саша', 'Соколова', 'по', 'шоссе', 'в', 'Краснодаре', 'мимо', 'театра', 'имени', 'Чехова']]
[['O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'I-ORG']]
<class 'list'>```

Для имен людей используются теги B-PER и I-PER (продолжение имени, если есть отчество или фамилия).

Случаются две основные ошибки: если субтокенов оказалось больше 512 (описано выше) или получилось несовпадение индексов (если попался пустой токен, то есть при разбиении сырого текста на предложения получился пустой элемент). 

In [8]:
# основная функция, извлекает именные сущности и фильтрует имена 

def names_extractor(tokenized_text): # функция получает токенизированный текст (список строк-предложений)
    names_list = [] # сюда будут капать только имена собственные из всех сущностей
    for item in tokenized_text: # без цикла не работает, если подать [tokenized_text] сразу ner, вернет пустой список
        try:
            tokens, tags = ner([item]) # квадратные скобки важны
            for tok, tag in zip(tokens[0], tags[0]): #zip позволяет итерироваться по неск объектам одновременно. Индексы важны, без них попадем не в тот уровень списка. 
                if tag == 'B-PER': # из всех найденных сущностей нам нужны только имена (без фамилий и отчеств, которые тегируются как I-PER)
                    names_list.append(tok)
        except RuntimeError as re: # если все-таки случится ошибка токенизации
            print("Got exception {}".format(repr(re)))
            print("Tokens-512 trouble with text:", tokenized_text, "\n\n")
            print("Tokens-512 trouble with items:", item, "\n\n")
        except IndexError as ie: # случается, если в поданном списке встретится пустой элемент (токен-предложение)
            print("Got exception {}".format(repr(ie))) # repr() is used to get a string representation of object (like str() but for development rather than for user)
            print("Index trouble with text:", tokenized_text, "\n\n")
            print("Index trouble with items:", item, "\n\n")
            
    return(names_list)
    print(names_list) 

In [341]:
names_list = names_extractor(f) # тестируем функцию

['Маша']

### Комментарий к функциям обновления датасета
1. Функция для создания списка списков имен, который дальше превратится в колонку датасета: в каждой ячейке колонки будет список имен, найденных в соотв. тексте, или пустой список, если имен в тексте нет. 

    Пустой список (NAMES) создается вне функции (в основном коде), иначе при запуске функции в цикле он будет пересоздаваться, а не пополняться. Но надо помнить, что при запуске функции для тестирования ячейку с командой создания списка также надо перезапускать - иначе все записанное в него до этого останется там.
    
2. Функция для бинарной проверки, были ли имена в тексте (да\нет), чтобы присвоить тексту категорию "адресный\неадресный - 1\0"

3. Функция для обновления татасета: создает две новых колонки

In [9]:
# функция для создания списка списков имен, который дальше превратится в колонку датасета

def big_list_appender(some_list): # получает список имен из одного текста
    NAMES.append(some_list) # NAMES создается вне функции
    return(NAMES) # вернет список списков

In [99]:
big_list_appender(names_list) # тестируем функцию

[['Ирина',
  'Денис',
  'Екатерина',
  'Люсю',
  'Уля',
  'Николь',
  'Николь',
  'Щенков',
  'Рыжий',
  'Ника',
  'Герда',
  'Мячик',
  'Арчи']]

In [100]:
NAMES

[['Ирина',
  'Денис',
  'Екатерина',
  'Люсю',
  'Уля',
  'Николь',
  'Николь',
  'Щенков',
  'Рыжий',
  'Ника',
  'Герда',
  'Мячик',
  'Арчи']]

In [10]:
# Классификация сборов на адресные и неадресные

def ad_checker(list_of_names_lists):
    for i in list_of_names_lists:
        if len(i) > 0:
            ADR.append(1) # ADR создается вне функции, чтобы не повторялось в цикле
        else:
            ADR.append(0)
    return(ADR)

In [103]:
ad_checker(NAMES) # тестируем функцию

[1]

In [11]:
def df_upd(some_df):
    some_df = some_df.assign(Names = NAMES)
    some_df = some_df.assign(Adr = ADR)
    return some_df

### Основной скрипт

In [35]:
%%time 

# %%time показывает время работы ячейки

if __name__ == "__main__": # в ipynb необязательная строка
    
    mydf = pd.read_csv('/Users/liza/PycharmProjects/Planeta_project/plset_fin_upd_clustered_rate.csv') # читаем датасет
    mydf.dropna(subset=["Description"], inplace=True) # выбрасываем пустые строки, если есть
    NAMES = [] # намеренно создается вне функции, см. выше
    ADR = []
    
    for text in mydf['Description'].values.astype('U'): # для проверки на первых n строках датасета можно перед двоеточием добавить [:n] (eg. [:15])
        k = sent_split(text) # применяем все функции к каждому тексту
        a = names_extractor(k)
        n = big_list_appender(a)
    
    t = ad_checker(n) #обязательно вне цикла, иначе циклы накладываются и список пополняется неадекватно 
   
    print(NAMES)
    print(ADR)
    print(len(NAMES))
    print(len(ADR))
    mydf1 = df_upd(mydf)
    mydf1.to_csv("plset_fin_upd_adr.csv")

[['Эгиды'], [], ['Зою', 'Ксану', 'Анну', 'Юрия', 'Фёдора', 'Алексея', 'Ilya'], ['Надежда', 'Кипиш', 'Островка', 'Маня', 'Хрюня', 'Люся', 'Джина', 'Кипиш', 'Кипиш', 'Кипиша'], [], [], ['Надежда', 'Островка', 'Островка'], [], [], [], ['Нелли', 'Надежда', 'Людмила', 'Мария', 'Елена', 'Ольга', 'Виктория'], [], ['Доктора', 'СУРГАНОВА', 'Александр', 'Юрий', 'ЮЮ', 'Артём', 'Роман', 'Романа', 'Алексей', 'Алексея', 'Алексей', 'Алексея', 'Антон', 'Антона', 'Алёна', 'Алёны', 'Константин', 'Константина', 'Михаил', 'Михаила', 'Андрей', 'Андрея', 'Вадим', 'Вадима', 'Максим', 'Максима', 'Николай', 'Николая', 'Андрей', 'Андрея', 'Алексей', 'Алексея', 'Алексея', 'Олег', 'Олега', 'Владимира', 'Владимира', 'Катя', 'Борис', 'Александр', 'Бориса', 'Александра', 'Андрей', 'Олег', 'Сева', 'КИНО', 'Александра', 'Георгия', 'Натальи', 'Александра', 'Георгия', 'Игоря', 'А', 'Георгия', 'Игоря', 'Виктора', 'Джоанны', 'Джоанны', 'Пётр', 'Валерия', 'А', 'Игорь', 'Андрей', 'Михаил', 'Евгений', 'Дмитрий', 'Константина

⬆︎ Примечание к выдаче %%time. If a computation requires two seconds of processor time, then two processors can (ideally) complete it in one second. Hence a two-processor system has two CPU seconds for every wall-clock second. Even if you do not use multi-threading explicitly in your process, a library you use or the operating system may use multiple processors to perform work for your process.

In [36]:
mydf1 # новые колонки Names и Adr добавились успешно

,Unnamed: 0,Title,Author,Description,Briefly,Goal,Result,Donations,Start,Finish,...,NumSentNLTK,Started,Finished,LeadBag,DaysLong,Cluster,Category,Rate,Names,Adr
0,0,Подарим жизнь,Бобруйское общественное объединение защиты жив...,"Друзья, рады приветствовать на странице нашего...",Спасение животных от эвтаназии на 5 сутки посл...,10000.0,76100,67,11 сентября 2017,9 ноября 2017,...,17,11 Sep 2017,9 Nov 2017,"['спасение', 'животное', 'эвтаназия', 'сутки',...",59,7,животные,1290.0,[Эгиды],1
1,1,Оплата нянечек для отказных детей в больницах,"Благотворительный фонд ""Дети без мам""","Когда ребёнок серьёзно болен, он ложится в бол...",Сбор средств на оплату ухода за детьми-сиротам...,150000.0,711665,470,1 июля 2014,16 сентября 2014,...,22,1 Jul 2014,16 Sep 2014,"['сбор', 'средство', 'оплата', 'уход', 'ребёно...",77,0,сироты_дети_из_неблагополучных семей,9240.0,[],0
2,2,Оплата нянь для детей-отказников в больницах,"Благотворительный фонд ""Дети без мам""","В больницах живут малыши, оставшиеся без попеч...",Сбор средств на уход за детьми-отказниками в б...,50000.0,236257,202,14 октября 2013,16 января 2014,...,20,14 Oct 2013,16 Jan 2014,"['сбор', 'средство', 'уход', 'ребёнок', 'отказ...",94,0,сироты_дети_из_неблагополучных семей,2510.0,"[Зою, Ксану, Анну, Юрия, Фёдора, Алексея, Ilya]",1
3,3,"""Не вешай нос! Или право на жизнь""",Местная общественная организация защитников жи...,"И снова здравствуйте, Друзья! \nМеня зовут Над...",Цель проекта - ликвидировать долги за ветерина...,29395.0,95300,170,21 февраля 2017,21 апреля 2017,...,24,21 Feb 2017,21 Apr 2017,"['цель', 'проект', 'ликвидировать', 'долг', 'в...",59,7,животные,1620.0,"[Надежда, Кипиш, Островка, Маня, Хрюня, Люся, ...",1
4,4,Календарь реабилитационного центра для птиц,Вера Пахомова,"Привет, друг! Мы снова на Planeta.ru!) \n В н...","""Воронье Гнездо"" собирает средства на печать к...",15000.0,42550,87,9 ноября 2019,8 декабря 2019,...,23,9 Nov 2019,8 Dec 2019,"['воронья', 'гнездо', 'собирать', 'средство', ...",29,4,активизм_просвещение_профилактика,1470.0,[],0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2033,2033,Поможем вместе двойняшкам Артемию и Арине!,БФ Милосердие,Благотворительный фонд помощи детям и инвалида...,7 декабря малышам исполнится 5 лет. Давайте сд...,120268.0,0,0,17 октября 2019,10 января 2020,...,29,17 Oct 2019,10 Jan 2020,"['декабрь', 'малыш', 'исполниться', 'год', 'да...",85,1,дети_лечение_реабилитация,0.0,"[Арине, Теме, Наталья, Арина, Артемий, Артемий...",1
2034,2034,Пункт кормления,СРОО Пункт Милосердия,Здравствуйте! Меня зовут Снежана! Мне 35 лет. ...,"В ""Пункте кормления"" готовятся и БЕСПЛАТНО раз...",333600.0,421,3,26 февраля 2020,31 марта 2020,...,51,26 Feb 2020,31 Mar 2020,"['пункт', 'кормление', 'готовиться', 'бесплатн...",34,4,бездомные_кризис,10.0,[Снежана],1
2035,2035,"SMG & Bees AWARDS 2020 Москва,Vegas City Hall","Благотворительный фонд ""АННА""",Друзья!\nУже в третий раз мы проводим междунар...,Bees Media Group проводит церемонию вручений п...,300000.0,0,0,22 декабря 2019,18 января 2020,...,15,22 Dec 2019,18 Jan 2020,"['проводить', 'церемония', 'вручение', 'премия...",27,4,активизм_просвещение_профилактика,0.0,"[Авраам, Татьяна, Наташа, Бьянка, Наргиз, Эдга...",1
2036,2036,Город мастеров: швейный цех 2.0,"АНО ""Открытый город""","Знакомьтесь, это мы - ""Город мастеров"" - инклю...","""Город мастеров"" - инклюзивные мастерские из Е...",200000.0,451,4,6 февраля 2020,15 апреля 2020,...,19,6 Feb 2020,15 Apr 2020,"['город', 'мастер', 'инклюзивный', 'мастерский...",69,12,социализация_возможности,10.0,[],0


# Общие выводы
Использованная модель хорошо справляется с извлечением именованных сущностей, но часто ошибается в классификации. Видно, что модель умеет ориентироваться на синтаксис. Например в строке:

> Запах костра, свет солнца сквозь ткань палатки, звон комаров, **Маша**, еда с дымком, гитарные переборы... **Маша** очень хочет в поход. Летний поход - это друзья, красивые места, купание, ягоды с куста и море впечатлений на весь следующий год. 

модель извлекает имя “Маша” только во втором использовании, то есть там, где это подлежащее. В обоих случаях имя добавлено искусственно.

В то же время, модель часто принимает название лекарств или организаций за имена людей. Из-за этого в разметке сборов на адресные и неадресные - много ошибок. Для нашей задачи это критично, поэтому для дальнейшего исследования поверх определения наличия имен в текстах с помощью DeepPavlov пришлось писать отдельный скрипт для более точной разметки (в другом ноутбуке). 

# Топонимы

In [165]:
mydf2 = pd.read_csv('/Users/liza/PycharmProjects/Planeta_project/plset_ver_011.csv') # датасет был обновлен в другой тетради, загружаю его, он более актуальный
mydf2 = mydf2.drop(mydf2.columns[0:5], axis=1) # удаляем лишние колонки с индексами
mydf2.dtypes # ок, колонки актуальные, лишних нет 

Title            object
Author           object
Description      object
Briefly          object
Goal            float64
Result            int64
Donations         int64
LengthZn          int64
LengthWd          int64
NumSent           int64
NumSentNLTK       int64
Started          object
Finished         object
LeadBag          object
DaysLong          int64
Cluster           int64
Category         object
Rate            float64
Names            object
Adr                bool
NamesSimRate      int64
dtype: object

In [175]:
mydf2.shape

(2031, 21)

In [167]:
# основная функция, извлекает именные сущности и фильтрует названия геогр. объектов 

def loc_extractor(tokenized_text): # функция получает токенизированный текст (список строк-предложений)
    loc_list = [] # сюда будут капать только геогр. названия из всех сущностей
    for item in tokenized_text: # без цикла не работает, если подать [tokenized_text] сразу в ner, вернет пустой список
        try:
            tokens, tags = ner([item]) # квадратные скобки важны
            for tok, tag in zip(tokens[0], tags[0]): #zip позволяет итерироваться по неск объектам одновременно. Индексы важны, без них попадем не в тот уровень списка. 
                if 'LOC' in tag: # из всех найденных сущностей нам нужны только топонимы (B-LOC и I-LOC)
                    loc_list.append(tok)
        except RuntimeError as re: # если все-таки случится ошибка токенизации
            print("Got exception {}".format(repr(re)))
            print("Tokens-512 trouble with text:", tokenized_text, "\n\n")
            print("Tokens-512 trouble with items:", item, "\n\n")
        except IndexError as ie: # случается, если в поданном списке встретится пустой элемент (токен-предложение)
            print("Got exception {}".format(repr(ie))) # repr() is used to get a string representation of object (like str() but for development rather than for user)
            print("Index trouble with text:", tokenized_text, "\n\n")
            print("Index trouble with items:", item, "\n\n")
            
    return(loc_list)
    print(loc_list)

In [168]:
# наполнение большого списка, который станет колонкой датасета
def loc_list_appender(some_list): # получает список названий из одного текста
    PLACES.append(some_list) # PLACES создается вне функции
    return(PLACES) # вернет список списков

In [177]:
# создание новой колонки 
def df_loc_upd(some_df): 
    some_df = some_df.assign(Loc = PLACES)
    return some_df

In [180]:
%%time 

if __name__ == "__main__": # в ipynb необязательная строка
    mydf2.dropna(subset=["Description"], inplace=True) # выбрасываем пустые строки, если есть
    PLACES = [] # намеренно создается вне функции, см. выше
    for text in mydf2['Description'].values.astype('U'): # для проверки на первых n строках датасета можно перед двоеточием добавить [:n] (eg. [:15])
        l = sent_split(text) # применяем все функции к каждому тексту
        o = loc_extractor(l)
        #print(o)
        c = loc_list_appender(o)
       
    print(PLACES)
    print(len(PLACES))
    mydf3 = df_loc_upd(mydf2)
    mydf3.to_csv("plset_ver_012.csv", index=False) # false - чтобы не записывалась колонка с индексами

[['Беларуси'], [], ['Нижнего', 'Новгорода'], ['Пермь', 'Островка', 'Островке', 'Пермяков', 'Тайга', 'Пермь'], [], ['Брянской', 'Московской', 'областях', 'Брянской', 'области'], ['Пермь', 'Островке', 'Пермь'], ['Волжском', 'Волгоградской', 'области', 'Волжского', 'Волгограда', 'Волжского', 'Волжского'], ['Москвы'], ['Челябинске', 'Южном', 'Урале'], ['Балашихи'], [], ['Москве', 'Индии', 'АКВАРИУМА', 'Индии', 'АКВАРИУМА', 'АЛИСА', 'АЛИСЫ', 'АЛИСЫ', 'АЛИСЫ', 'Харькове', 'Донецка', 'Украине', 'Россию', 'Белгороде', 'Москву', 'Израиля', 'Курск'], ['Реммаш', 'Сергиево', '-', 'Посадского', 'района', 'Москвы', 'Люберецком', 'районе', 'Петровско', '-', 'Разумовской', 'России'], ['Кислухинскому', 'заказнику', 'Кислухинского', 'заказника', 'Алтайском', 'крае', 'СССР', 'Кислухинском', 'заказнике', 'Среднеобского', 'бора', 'Оби', 'России', 'Кислухинского', 'заказника', 'Кислухинский', 'лес', 'России', 'России', 'Кислухинского', 'леса'], ['Москву'], ['Земля'], ['России'], ['Кузбассе', 'Кузбассе', 'Ку

In [211]:
mydf3.loc[13]['Loc'] # полностью посмотреть содержимое ячейки

['Реммаш',
 'Сергиево',
 '-',
 'Посадского',
 'района',
 'Москвы',
 'Люберецком',
 'районе',
 'Петровско',
 '-',
 'Разумовской',
 'России']

### Чистка полученных названий и создание списка для визуализации на карте

DeepPavlov составные названия разбивает на разные токены. Ниже - муторное, но рабочее решение обработки данных для дальнейшей работы. 

In [184]:
pip install pymystem3 # в этой среде не был уставновлен

  Using cached pymystem3-0.2.0-py3-none-any.whl (10 kB)
Note: you may need to restart the kernel to use updated packages.


In [189]:
import re
import ast
from nltk.tokenize import word_tokenize, wordpunct_tokenize # попробуем две функции
from pymystem3 import Mystem
mystem = Mystem() 

In [186]:
# список частых элементов составных названий

double = ['великий', 'нижний', 'верхний', 'старый', 'новый', 'ближний', 'дальний', 'южный', 'вышний', 'кривой', 'белый', 'красный', 'павловский', 'сергиев', 
          'набережный', 'сосновый']

In [309]:
# функция для обработки того, что DeepPavlov извлек 

def prepr(lst): # получает список названий из ячейки датасета
    #print(lst)
    str = " ".join(lst) # превращает его в строку для применения строковых методов дальше
    #print(str)
    str = re.sub(r' - ', "-", str) # сшиваем составные названия типа Ростов-на-Дону, re.sub ловит не все тире, но некоторые ловит :(
    str = " ".join(mystem.lemmatize(str)) # лемматизация. mystem возвращает с пустыми строками, поэтому join
    str = str.strip() # mystem пишет \n в конце списка, удаляем
    str = str.split() # снова превращаем строку в список
    #print(str)
    a = [] # сюда пойдут города после объединения составных названий
    for i in range(len(str)):
        if str[i] in double: # сверяемся со списком приставок, который задали выше
            #print(str[i])
            new_str = f'{str[i]} {str[i+1]}' # объединяем составные названия
            str[i+1] = "substitute" # после объединения вторая часть остается, а если ее удалять, меняется длина списка и зависящая от нее индексация
            #print(new_str)
            a.append(new_str) # пишем в список
            #print(a)
        else:
            a.append(str[i]) # если название не совпало с приставкой, просто пишем его в тот же список
    a = set(a) # убираем дубли
    if "substitute" in a:
        a.remove("substitute") # удаляем замену    
    print(a) 
    return(a) # получаем список уникальных объектов в нормальной форме 

In [310]:
prepr(['Вышнего', 'Волочка', 'Ростов', '-', 'на', '-', 'Дону', 'Набережных', 'Челнах'])

{'набережная', 'вышний волочек', 'ростов-на-дону', 'челны'}


{'вышний волочек', 'набережная', 'ростов-на-дону', 'челны'}

In [245]:
del PLACES_new # на всякий случай чистим переменную

In [246]:
%%time 

if __name__ == "__main__": # в Jupyter необязательная строка
    
    PLACES_new = [] # станет колонкой датасета
    mydf4 = mydf3
    for loc_list in mydf3['Loc']: # в этой колонке списки мест, извлеченных раньше
        PLACES_new.append(prepr(loc_list))
    mydf4['Loc'] = PLACES_new # перезаписываем колонку

{'беларусь'}
set()
{'нижний новгород'}
{'пермь', 'пермяков', 'тайга', 'островок'}
set()
{'московский', 'область', 'брянский'}
{'пермь', 'островок'}
{'волгоград', 'область', 'волжский', 'волгоградский'}
{'москва'}
{'челябинск', 'южный урал'}
{'балашиха'}
set()
{'алиса', 'индия', 'харьков', 'израиль', 'белгород', 'курск', 'аквариум', 'россия', 'донецк', 'москва', 'украина'}
{'район', 'разумовский', 'петровский', 'люберецкий', '-', 'реммаш', 'россия', 'москва', 'сергиево-посадский'}
{'обь', 'край', 'лес', 'среднеобский', 'кислухинский', 'ссср', 'заказник', 'алтайский', 'россия', 'бор'}
{'москва'}
{'земля'}
{'россия'}
{'кемерово', 'иркутск', 'кузбасс'}
{'москва'}
{'москва'}
{'душа'}
{'москва', 'тушино', 'район', 'одинцовский'}
{'ветлекарь', 'проспект', 'новосибирск', 'красный'}
set()
{'вологодский', 'область'}
{'сургут'}
{'шамарин', 'калуга', 'рф'}
{'дубно', 'россия'}
{'санкт-петербург', 'икея'}
set()
set()
set()
{'алтай', 'новосибирск', 'гора', 'алтайский', 'москва', 'актр'}
{'башкортоста

In [247]:
mydf4 # ок, все переписалось

,Title,Author,Description,Briefly,Goal,Result,Donations,LengthZn,LengthWd,NumSent,...,Finished,LeadBag,DaysLong,Cluster,Category,Rate,Names,Adr,NamesSimRate,Loc
0,Подарим жизнь,Бобруйское общественное объединение защиты жив...,"Друзья, рады приветствовать на странице нашего...",Спасение животных от эвтаназии на 5 сутки посл...,10000.0,76100,67,1134,193,17,...,9 Nov 2017,"['спасение', 'животное', 'эвтаназия', 'сутки',...",59,7,животные,1290.0,['Эгиды'],False,0,{беларусь}
1,Оплата нянечек для отказных детей в больницах,"Благотворительный фонд ""Дети без мам""","Когда ребёнок серьёзно болен, он ложится в бол...",Сбор средств на оплату ухода за детьми-сиротам...,150000.0,711665,470,1749,300,21,...,16 Sep 2014,"['сбор', 'средство', 'оплата', 'уход', 'ребёно...",77,0,сироты_дети_из_неблагополучных семей,9240.0,[],False,0,{}
2,Оплата нянь для детей-отказников в больницах,"Благотворительный фонд ""Дети без мам""","В больницах живут малыши, оставшиеся без попеч...",Сбор средств на уход за детьми-отказниками в б...,50000.0,236257,202,1693,275,20,...,16 Jan 2014,"['сбор', 'средство', 'уход', 'ребёнок', 'отказ...",94,0,сироты_дети_из_неблагополучных семей,2510.0,"['Зою', 'Ксану', 'Анну', 'Юрия', 'Фёдора', 'Ал...",False,11,{нижний новгород}
3,"""Не вешай нос! Или право на жизнь""",Местная общественная организация защитников жи...,"И снова здравствуйте, Друзья! \nМеня зовут Над...",Цель проекта - ликвидировать долги за ветерина...,29395.0,95300,170,2138,383,33,...,21 Apr 2017,"['цель', 'проект', 'ликвидировать', 'долг', 'в...",59,7,животные,1620.0,"['Надежда', 'Кипиш', 'Островка', 'Маня', 'Хрюн...",False,35,"{пермь, пермяков, тайга, островок}"
4,Календарь реабилитационного центра для птиц,Вера Пахомова,"Привет, друг! Мы снова на Planeta.ru!) \n В н...","""Воронье Гнездо"" собирает средства на печать к...",15000.0,42550,87,1765,340,24,...,8 Dec 2019,"['воронья', 'гнездо', 'собирать', 'средство', ...",29,4,активизм_просвещение_профилактика,1470.0,[],False,0,{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026,Поможем вместе двойняшкам Артемию и Арине!,БФ Милосердие,Благотворительный фонд помощи детям и инвалида...,7 декабря малышам исполнится 5 лет. Давайте сд...,120268.0,0,0,1952,346,29,...,10 Jan 2020,"['декабрь', 'малыш', 'исполниться', 'год', 'да...",85,1,дети_лечение_реабилитация,0.0,"['Арине', 'Теме', 'Наталья', 'Арина', 'Артемий...",False,42,{россия}
2027,Пункт кормления,СРОО Пункт Милосердия,Здравствуйте! Меня зовут Снежана! Мне 35 лет. ...,"В ""Пункте кормления"" готовятся и БЕСПЛАТНО раз...",333600.0,421,3,4016,737,65,...,31 Mar 2020,"['пункт', 'кормление', 'готовиться', 'бесплатн...",34,4,бездомные_кризис,10.0,['Снежана'],False,0,"{тольятти, всегда, чечня, афганистан, общество}"
2028,"SMG & Bees AWARDS 2020 Москва,Vegas City Hall","Благотворительный фонд ""АННА""",Друзья!\nУже в третий раз мы проводим междунар...,Bees Media Group проводит церемонию вручений п...,300000.0,0,0,1453,256,19,...,18 Jan 2020,"['проводить', 'церемония', 'вручение', 'премия...",27,4,активизм_просвещение_профилактика,0.0,"['Авраам', 'Татьяна', 'Наташа', 'Бьянка', 'Нар...",False,24,"{ситти, вегас, холл}"
2029,Город мастеров: швейный цех 2.0,"АНО ""Открытый город""","Знакомьтесь, это мы - ""Город мастеров"" - инклю...","""Город мастеров"" - инклюзивные мастерские из Е...",200000.0,451,4,1734,289,27,...,15 Apr 2020,"['город', 'мастер', 'инклюзивный', 'мастерский...",69,12,социализация_возможности,10.0,[],False,0,{екатеринбург}


In [248]:
mydf4.to_csv("plset_ver_013.csv", index=False) # сохраняем новый датасет в файл без индексов

In [249]:
print(PLACES_new)
type(PLACES_new)

[{'беларусь'},
 set(),
 {'нижний новгород'},
 {'островок', 'пермь', 'пермяков', 'тайга'},
 set(),
 {'брянский', 'московский', 'область'},
 {'островок', 'пермь'},
 {'волгоград', 'волгоградский', 'волжский', 'область'},
 {'москва'},
 {'челябинск', 'южный урал'},
 {'балашиха'},
 set(),
 {'аквариум',
  'алиса',
  'белгород',
  'донецк',
  'израиль',
  'индия',
  'курск',
  'москва',
  'россия',
  'украина',
  'харьков'},
 {'-',
  'люберецкий',
  'москва',
  'петровский',
  'разумовский',
  'район',
  'реммаш',
  'россия',
  'сергиево-посадский'},
 {'алтайский',
  'бор',
  'заказник',
  'кислухинский',
  'край',
  'лес',
  'обь',
  'россия',
  'среднеобский',
  'ссср'},
 {'москва'},
 {'земля'},
 {'россия'},
 {'иркутск', 'кемерово', 'кузбасс'},
 {'москва'},
 {'москва'},
 {'душа'},
 {'москва', 'одинцовский', 'район', 'тушино'},
 {'ветлекарь', 'красный', 'новосибирск', 'проспект'},
 set(),
 {'вологодский', 'область'},
 {'сургут'},
 {'калуга', 'рф', 'шамарин'},
 {'дубно', 'россия'},
 {'икея', '

PLACES_new - это список множеств (set), в том числе пустых. Извлечем оттуда факты. 

In [269]:
points = []
for a_set in PLACES_new:
    if len(a_set) > 0: # если не пустой
        #print(a_set)
        for item in a_set:
            points.append(item) 
points = set(points)
print(len(points))
print(points) # получили множество уникальных элементов

1530
{'нижний новогород', 'мыс', 'буденновский', 'лагонакский', 'рыбинск', 'город', 'гунда', 'мегет', 'удмуртский', 'смирновский', 'лос-анджелес', 'армения', 'р', 'жуковский', 'творец', 'башкирия', 'туго', 'петродворец', 'абрамцевский', 'о', 'москва', 'климов', 'лондон', 'радуга', 'спасать', 'сердобск', 'ленинский', 'садовый', 'чапаевск', 'северный', 'огарковый', 'кинешма', 'сибирский', 'угличский', 'тихорецкий', 'истра', 'пуся', 'новый оскол', 'переулок', 'звенигород', 'пошехонье', 'ивановский', 'куршский', 'гороховый', 'шнайдер', 'мстера', 'приморский', 'курай', 'север', 'салехард', 'красноляжский', 'столица', 'котовск', 'павловск', 'ангарск', 'Иркутскй', 'Park', 'клемен', 'одинцовский', 'канада', 'зао', 'берлин', 'брянский', 'сборная', 'балтийск', 'генуэзский', 'тулун', 'южный подмосковье', 'бокситогорский', 'салаватский', 'сахар', 'сожаление', 'элиста', 'улан-удэ', 'юкаменское', 'вепсский', 'хаб', 'токио', 'стерлитамак', 'как', 'челобитьево', 'березняк', 'хакасия', 'шампунь', 'вели

In [278]:
type(points)
print(points) # тут не только топонимы, т.к. из текстов извлеклось иного лишнего

{'нижний новогород',
 'мыс',
 'буденновский',
 'лагонакский',
 'рыбинск',
 'город',
 'гунда',
 'мегет',
 'удмуртский',
 'смирновский',
 'лос-анджелес',
 'армения',
 'р',
 'жуковский',
 'творец',
 'башкирия',
 'туго',
 'петродворец',
 'абрамцевский',
 'о',
 'москва',
 'климов',
 'лондон',
 'радуга',
 'спасать',
 'сердобск',
 'ленинский',
 'садовый',
 'чапаевск',
 'северный',
 'огарковый',
 'кинешма',
 'сибирский',
 'угличский',
 'тихорецкий',
 'истра',
 'пуся',
 'новый оскол',
 'переулок',
 'звенигород',
 'пошехонье',
 'ивановский',
 'куршский',
 'гороховый',
 'шнайдер',
 'мстера',
 'приморский',
 'курай',
 'север',
 'салехард',
 'красноляжский',
 'столица',
 'котовск',
 'павловск',
 'ангарск',
 'Иркутскй',
 'Park',
 'клемен',
 'одинцовский',
 'канада',
 'зао',
 'берлин',
 'брянский',
 'сборная',
 'балтийск',
 'генуэзский',
 'тулун',
 'южный подмосковье',
 'бокситогорский',
 'салаватский',
 'сахар',
 'сожаление',
 'элиста',
 'улан-удэ',
 'юкаменское',
 'вепсский',
 'хаб',
 'токио',
 'ст

Чтобы в списке остались только настоящие названия городов (без "Какао", "РФ", "Бирюлево"), сравним их со списком городов РФ (их всего чуть более 1000). Из текстов извеклись названия других стран и городов в других странах. Но пока для упрощения задачи, посмотрим на охват проектов по России. В целом это оправданно, т.к. planeta.ru - русскоязычная площадка, большинство организаций здесь - российские.

In [297]:
cities = pd.read_csv('/Users/liza/PycharmProjects/Planeta_project/cities_rf.csv', sep = ';') # список городов РФ из интернета, чуть-чуть обработанный
cities #разделителем оказалось ";" - это можно проверить, если открыть файл без уточнения разделителя и вывести head

,SNo,City,Region,FederalDistrict,Population,Set
0,1,Адыгейск,Адыгея,Южный,12248,1973
1,2,Майкоп,Адыгея,Южный,144246,1857
2,3,Горно-Алтайск,Алтай,Сибирский,56928,1830
3,4,Алейск,Алтайский край,Сибирский,29512,1913
4,5,Барнаул,Алтайский край,Сибирский,612091,1730
...,...,...,...,...,...,...
1104,1105,Ростов,Ярославская область,Центральный,31791,912
1105,1106,Рыбинск,Ярославская область,Центральный,200771,1071
1106,1107,Тутаев,Ярославская область,Центральный,41001,13 век
1107,1108,Углич,Ярославская область,Центральный,34505,1148


In [303]:
rus_cities = [city.lower() for city in cities.City]
rus_cities # получили список, с которым будем сравнивать 

['адыгейск',
 'майкоп',
 'горно-алтайск',
 'алейск',
 'барнаул',
 'белокуриха',
 'бийск',
 'горняк',
 'заринск',
 'змеиногорск',
 'камень-на-оби',
 'новоалтайск',
 'рубцовск',
 'славгород',
 'яровое',
 'белогорск',
 'благовещенск',
 'завитинск',
 'зея',
 'райчихинск',
 'свободный',
 'сковородино',
 'тында',
 'шимановск',
 'архангельск',
 'вельск',
 'каргополь',
 'коряжма',
 'котлас',
 'мезень',
 'мирный',
 'новодвинск',
 'няндома',
 'онега',
 'северодвинск',
 'сольвычегодск',
 'шенкурск',
 'астрахань',
 'ахтубинск',
 'знаменск',
 'камызяк',
 'нариманов',
 'харабали',
 'агидель',
 'баймак',
 'белебей',
 'белорецк',
 'бирск',
 'благовещенск',
 'давлеканово',
 'дюртюли',
 'ишимбай',
 'кумертау',
 'межгорье',
 'мелеуз',
 'нефтекамск',
 'октябрьский',
 'салават',
 'сибай',
 'стерлитамак',
 'туймазы',
 'уфа',
 'учалы',
 'янаул',
 'алексеевка',
 'белгород',
 'бирюч',
 'валуйки',
 'грайворон',
 'губкин',
 'короча',
 'новый оскол',
 'старый оскол',
 'строитель',
 'шебекино',
 'брянск',
 'дятько

In [306]:
pl_cities = [point for point in points if point in rus_cities]
print(len(pl_cities)) # осталось 284 записи из 1500, т.к. было очень много мусора
# pl_cities

284


In [307]:
# пишем чистый список в файл
import csv
with open('pl_cities.csv', 'w', newline='') as csvfile: # newline='' нужен по документации
    pointwriter = csv.writer(csvfile, dialect='excel') # разделитесь по дефолту запятая, можно указать лиалект excel
    for element in pl_cities: # вне цикла, если просто подать points или [points], пишет каждую букву в отд. колонку или все в одну строку
        pointwriter.writerow([element])

Мы получили чистый список уникальных нормализованных названий российский городов, которые упоминаются в текстах датасета. И теперь можем визуализировать на карте геогр. охват благотворительных сборов на площадке. Это можно сделать в Python и R, но для этого нужны координаты городов. Можно найти данные в интернете и добавить в базу, а можно загрузить список-файл в google maps. Из 284 объектов списка google не смог найти 18. Еще несколько городов (видимо, тезок российских) все же оказались в других странах. Жалко, но терпимо, учитывая, что это быстрое и удобное решение для общего представления, не требующее мороки с координатами. 

[Посмотреть карту можно здесь](https://www.google.com/maps/d/drive?state=%7B%22ids%22%3A%5B%221QO6ifQ9O7HUoU7ZnqNabLjptXXmhMU0_%22%5D%2C%22action%22%3A%22open%22%2C%22userId%22%3A%22102543131183110556280%22%7D&usp=sharing)

## Подвал (рабочее)
Тесты и неиспользованные ячейки, оставляю для себя.

In [121]:
str = "Шла Саша Соколова по шоссе в Нижнем Новгороде мимо театра имени Чехова. Петя тоже шел мимо."

In [156]:
PLACES = []
l = sent_split(str) # применяем все функции к каждому тексту
o = loc_extractor(l)
c = loc_list_appender(o)

In [152]:
l = sent_split(str) # применяем все функции к каждому тексту
l

['Шла Саша Соколова по шоссе в Нижнем Новгороде мимо театра имени Чехова.',
 'Петя тоже шел мимо.']

In [153]:
o = loc_extractor(l)
o

['Нижнем', 'Новгороде']

In [79]:
ner("Мама мыла раму") # здесь будет ошибка из отсутствия []

IndexError: index 0 is out of bounds for axis 0 with size 0

In [78]:
ner([["Мама мыла раму"],['Шла Саша по шоссе']])

[[['Мама мыла раму'], ['Шла Саша по шоссе']], [['O'], ['O']]]

In [70]:
tokens, tags = ner([["Мама мыла раму"],['Шла Саша по шоссе']])
tokens, tags

([['Мама мыла раму'], ['Шла Саша по шоссе']], [['O'], ['O']])

In [106]:
tokens, tags = ner(["Шла Саша Соколова по шоссе в Нижнем Новгороде мимо театра имени Чехова. Петя тоже шел мимо."])

In [107]:
for tok, tag in zip(tokens[0], tags[0]): #zip позволяет итерироваться по неск объектам одновременно. Индексы важны, без них попадем не в тот уровень списка. 
    print(f'{tok}\t\t{tag}') #  литерал f - один из способов (самы совр.) форматирования строк, альтернатива format, $, % и др.

Шла		O
Саша		B-PER
Соколова		I-PER
по		O
шоссе		O
в		O
Нижнем		B-LOC
Новгороде		I-LOC
мимо		O
театра		B-ORG
имени		I-ORG
Чехова		I-ORG
.		O
Петя		O
тоже		O
шел		O
мимо		O
.		O


In [87]:
print(tokens)
print(tags)
print(type(tokens))

[['Шла', 'Саша', 'Соколова', 'по', 'шоссе', 'в', 'Краснордаре', 'мимо', 'театра', 'имени', 'Чехова']]
[['O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'I-ORG', 'I-ORG']]
<class 'list'>


In [ ]:
from deeppavlov.models.preprocessors.odqa_preprocessors import DocumentChunker 

# альтернатива токенизаторам по предложениям. плюсы: можно задать макс. длину токена с соблюдением длины предложениях. минусы: модель училась на предложениях, лучше давать ей предложения, а не куски

In [ ]:
chunker = DocumentChunker(tokens_limit = 200, flatten_result = True)

In [ ]:
chunks = chunker([text3])[0]
print(chunks, len(chunks))

In [ ]:
chs = [ch for ch in chunks]
chs